In [1]:
import pandas as pd

# merge files
df1 = pd.read_csv('Youtube01-Psy.csv')
df2 = pd.read_csv('Youtube02-KatyPerry.csv')
df3 = pd.read_csv('Youtube03-LMFAO.csv')
df4 = pd.read_csv('Youtube04-Eminem.csv')
df5 = pd.read_csv('Youtube05-Shakira.csv')

df = pd.concat([df1, df2, df3, df4, df5])
df.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [2]:
df.drop(['COMMENT_ID', 'AUTHOR', 'DATE'], axis=1, inplace=True)

df.rename(columns={'CLASS': 'label', 'CONTENT': 'text'}, inplace=True)
df["label"].value_counts()

1    1005
0     951
Name: label, dtype: int64

In [3]:
df.isna().sum()

text     0
label    0
dtype: int64

In [12]:
sms = pd.read_csv("SMSSpamCollection.txt", sep='\t', lineterminator='\n', header=None)

In [13]:
sms.head()

,0,1
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [14]:
sms.shape

(5572, 2)

In [15]:
sms.columns = ['label', 'text']

sms['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [16]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
sms['label'] = le.fit_transform(sms['label'])

sms["label"].value_counts()

0    4825
1     747
Name: label, dtype: int64

In [17]:
df = pd.concat([df, sms])

In [19]:
df["label"].value_counts()

0    5776
1    1752
Name: label, dtype: int64

In [20]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
import re

# Download the required NLTK resources
nltk.download("punkt")
nltk.download("wordnet")

lemmatizer = WordNetLemmatizer()

# Function to lemmatize text
def custom_lemmatizer(input_text):
    # Tokenize the input text into individual words
    words_list = word_tokenize(input_text)

    # Create a list to store lemmatized words
    lemmatized_list = []

    # Iterate through each word and its POS tag
    for word, tag in pos_tag(words_list):
        # Determine the WordNet tag from the first character of the POS tag
        wn_tag = tag[0].lower() if tag[0].lower() in ['a', 'r', 'n', 'v'] else None

        # Lemmatize the word using the determined WordNet tag
        lemma_word = lemmatizer.lemmatize(word, wn_tag) if wn_tag else word

        # Append the lemmatized word to the list
        lemmatized_list.append(lemma_word)

    # Join the lemmatized words into a string and return
    return " ".join(lemmatized_list)


# Function to remove special characters from the text
def remove_special_char(text):
    if not isinstance(text, str):
        return str(text)  # Convert non-string data to string
    
    pattern = r"[^a-zA-Z0-9\s]"
    cleaned_text = re.sub(pattern, "", text)
    return cleaned_text


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Furkan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Furkan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [21]:
df['text'] = df['text'].str.lower()

df["cleaned-text"] = df["text"].apply(remove_special_char)
df["cleaned-text"] = df["cleaned-text"].apply(custom_lemmatizer)

In [22]:
df.head()

,text,label,cleaned-text
0,"huh, anyway check out this you[tube] channel: ...",1,huh anyway check out this youtube channel koby...
1,hey guys check out my new channel and our firs...,1,hey guy check out my new channel and our first...
2,just for test i have to say murdev.com,1,just for test i have to say murdevcom
3,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1,me shake my sexy as on my channel enjoy
4,watch?v=vtarggvgtwq check this out .﻿,1,watchvvtarggvgtwq check this out


In [23]:
df.drop(['text'], axis=1, inplace=True)

In [25]:
# we will do Naive Bayes, Random Forest and SVM 
from sklearn.model_selection import train_test_split

X = df['cleaned-text']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

nb = MultinomialNB()
nb.fit(X_train, y_train)

y_pred = nb.predict(X_test)

print("Accuracy score: ", accuracy_score(y_test, y_pred))

print("Confusion matrix: \n", confusion_matrix(y_test, y_pred))

print("Classification report: \n", classification_report(y_test, y_pred))

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

print("Accuracy score: ", accuracy_score(y_test, y_pred))

print("Confusion matrix: \n", confusion_matrix(y_test, y_pred))

print("Classification report: \n", classification_report(y_test, y_pred))

from sklearn.svm import SVC

svc = SVC()
svc.fit(X_train, y_train)

y_pred = svc.predict(X_test)

print("Accuracy score: ", accuracy_score(y_test, y_pred))

print("Confusion matrix: \n", confusion_matrix(y_test, y_pred))

print("Classification report: \n", classification_report(y_test, y_pred))

Accuracy score:  0.899734395750332
Confusion matrix: 
 [[1132    2]
 [ 149  223]]
Classification report: 
               precision    recall  f1-score   support

           0       0.88      1.00      0.94      1134
           1       0.99      0.60      0.75       372

    accuracy                           0.90      1506
   macro avg       0.94      0.80      0.84      1506
weighted avg       0.91      0.90      0.89      1506

Accuracy score:  0.949535192563081
Confusion matrix: 
 [[1129    5]
 [  71  301]]
Classification report: 
               precision    recall  f1-score   support

           0       0.94      1.00      0.97      1134
           1       0.98      0.81      0.89       372

    accuracy                           0.95      1506
   macro avg       0.96      0.90      0.93      1506
weighted avg       0.95      0.95      0.95      1506

Accuracy score:  0.952191235059761
Confusion matrix: 
 [[1128    6]
 [  66  306]]
Classification report: 
               precision  

### Naive Bayes Failed here however others got a better result

In [31]:
# Ensemble model
from sklearn.ensemble import VotingClassifier

vc = VotingClassifier(estimators=[('nb', nb), ('rf', rf), ('svc', svc)], voting='hard')
vc.fit(X_train, y_train)

y_pred = vc.predict(X_test)

print("Accuracy score: ", accuracy_score(y_test, y_pred))

print("Confusion matrix: \n", confusion_matrix(y_test, y_pred))

print("Classification report: \n", classification_report(y_test, y_pred))

Accuracy score:  0.9475431606905711
Confusion matrix: 
 [[1132    2]
 [  77  295]]
Classification report: 
               precision    recall  f1-score   support

           0       0.94      1.00      0.97      1134
           1       0.99      0.79      0.88       372

    accuracy                           0.95      1506
   macro avg       0.96      0.90      0.92      1506
weighted avg       0.95      0.95      0.95      1506



In [33]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.load_state_dict(torch.load('bert_model.pt'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# predict youtube and sms spam

train_text, temp_text, train_labels, temp_labels = train_test_split(df['cleaned-text'], df['label'], 
                                                                    random_state=2018, 
                                                                    test_size=0.1, 
                                                                    stratify=df['label'])

# We'll further split the validation set into validation and test datasets
val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=2018, 
                                                                test_size=0.3, 
                                                                stratify=temp_labels)

test_encodings = tokenizer(test_text.tolist(), truncation=True, padding=True, max_length=256)

# Convert inputs to PyTorch tensors
test_inputs = torch.tensor(test_encodings['input_ids'])
test_labels = torch.tensor(test_labels.tolist())
test_masks = torch.tensor(test_encodings['attention_mask'])

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=16)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.load_state_dict(torch.load('bert_model.pt'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
import numpy as np

# Assuming you have test_dataloader defined and ready to use

def evaluate_test_set(test_dataloader, top_n):
    predictions = []
    true_labels = []
    correct_predictions = 0

    for batch in test_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        # Tell the model not to compute or store gradients
        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

        logits = outputs.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Get the top N predictions with probabilities for each sample in the batch
        top_n_indices = np.argsort(logits, axis=1)[:, -top_n:][:, ::-1]
        top_n_probs = np.take_along_axis(logits, top_n_indices, axis=1)

        # Convert the predicted indices and probabilities to their corresponding labels
        batch_predictions = []
        for sample_probs, sample_indices in zip(top_n_probs, top_n_indices):
            sample_predictions = [
                (label_id, prob) for label_id, prob in zip(sample_indices, sample_probs)
            ]
            batch_predictions.append(sample_predictions)

        predictions.extend(batch_predictions)
        true_labels.extend(label_ids)

        # Check if the true label is in the top N predictions
        for sample_preds, true_label_id in zip(batch_predictions, label_ids):
            if true_label_id in [pred_label_id for pred_label_id, _ in sample_preds]:
                correct_predictions += 1

    total_samples = len(test_dataloader.dataset)
    accuracy = correct_predictions / total_samples

    return predictions, true_labels, accuracy

# Call the function to get the top 3 predictions with probabilities and calculate accuracy
predictions, true_labels, accuracy = evaluate_test_set(test_dataloader, top_n=1)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
print(accuracy)

0.8495575221238938


# Without training the model specialized for the task, BERT cannot see ahead. BERT is powerful within the specific borders drawn.